# Obtain details of edge circadian score in each path

In [1]:
import numpy as np
import pandas as pd
import src.calculate_edge_circa_score as cal
import requests

In [2]:
# read in processed CircaDB data 
circa_db = pd.read_csv('data/circa_db_mapped.tsv', sep = '\t')
circa_db.head()

,gene_id,Fat SQ_fdr,Fat Visceral_fdr,Aorta_fdr,Artery Coronary_fdr,Artery Tibial_fdr,Colon_fdr,Esophagus_fdr,Heart Atrial_fdr,Liver_fdr,...,Artery Coronary_exp,Artery Tibial_exp,Colon_exp,Esophagus_exp,Heart Atrial_exp,Liver_exp,Lung_exp,Nerve Tibial_exp,Pituitary_exp,Thyroid_exp
0,653635,0.6432,0.1526,0.8443,0.7712,0.9549,0.5059,0.2928,0.6953,0.9732,...,12.300,11.590,12.7200,12.3033,5.369,5.406,13.680,19.480,15.840,19.2550
1,79854,0.7652,0.2412,0.0473,0.0002,0.6314,0.8602,0.0820,0.6481,0.6405,...,5.940,8.419,5.4407,4.6607,2.962,3.240,7.024,12.110,9.898,9.7615
2,643837,0.9075,0.0774,0.7887,0.2173,0.6017,0.3294,0.0696,0.2758,0.3438,...,9.039,11.520,7.1738,7.6350,14.350,3.674,6.016,5.872,18.600,6.5845
3,26155,0.8656,0.4570,0.5405,0.6391,0.8885,0.3902,0.6801,0.8984,0.3532,...,52.240,62.410,50.8450,62.9550,30.670,28.420,57.320,69.660,57.560,66.5800
4,339451,0.8808,0.6235,0.4360,0.4999,0.7446,0.7780,0.6238,0.6645,0.9089,...,13.510,10.800,11.6200,14.5633,4.870,5.979,15.600,9.224,21.420,15.4450


In [3]:
# read in metapath data
metapath_df = pd.read_csv('data/rephetio_significant_metapaths.tsv', header = 0, sep = '\t')
metapath_df.head()

,abbreviation,length,delta_auroc,negative_log_p
0,CtDaGaD,3,12.70%,7.2485
1,CbGbCtD,3,21.70%,6.4962
2,CbGaD,2,14.50%,6.2137
3,CrCbGaD,3,8.22%,6.0926
4,CbGeAlD,3,8.43%,5.1873


In [4]:
metapaths = list(metapath_df.abbreviation)

## Example 1: flurbiprofen (DB00712) and rheumatoid arthritis (DOID:7148). 
This pair has score of 1 in liver, and 0 in all the other 12 tissues. 

In [5]:
# Return score details of Flurbiprofen ~ rheumatoid arthritis
drug = 'DB00712'
disease = 'DOID:7148'
tissues = ['Aorta','Artery Coronary','Artery Tibial','Colon','Esophagus','Fat SQ','Fat Visceral','Heart Atrial','Liver',
           'Lung','Nerve Tibial','Pituitary','Thyroid']
score = cal.detail_edge_circa_score(drug, disease, tissues, circa_db, query_metapath = metapaths)

In [6]:
# edge circadian score of drug-disease pair in 13 tissues
score['edge_circa_score']

{'Aorta': 0.0,
 'Artery Coronary': 0.0,
 'Artery Tibial': 0.0,
 'Colon': 0.0,
 'Esophagus': 0.0,
 'Fat SQ': 0.0,
 'Fat Visceral': 0.0,
 'Heart Atrial': 0.0,
 'Liver': 1.0,
 'Lung': 0.0,
 'Nerve Tibial': 0.0,
 'Pituitary': 0.0,
 'Thyroid': 0.0}

In [7]:
# number of metapaths that contain gene
score['total_meta_count']

1

In [8]:
# number of paths that contain gene in CircaDB
score['total_path_count']

12

In [9]:
# each row represents a path
score['score_details'].head(2)

,source_node,target_node,metapath,node_ids,node_names,rel_ids,rel_names,gene_symbol,whether_in_circadb,circadian_tissue,non_circadian_tissue,importance_score
0,Flurbiprofen,rheumatoid arthritis,CbGbCpD,"16339,6261,37708,31624","Flurbiprofen,PTGS2,Salsalate,rheumatoid arthritis","1042317,1757362,326840","BINDS_CbG,BINDS_CbG,PALLIATES_CpD",PTGS2,False,NaN,NaN,1.696160
1,Flurbiprofen,rheumatoid arthritis,CbGbCpD,"16339,36580,37708,31624","Flurbiprofen,PTGS1,Salsalate,rheumatoid arthritis","901577,302279,326840","BINDS_CbG,BINDS_CbG,PALLIATES_CpD",PTGS1,True,Liver,"Aorta,Artery Coronary,Artery Tibial,Colon,Esop...",1.373082


For paths that traverse a gene not in CircaDB, the 'whether_in_circadb' column will be False. Both 'circadian_tissue' and 'non_circadian_tissue' columns will be NaN. These paths will not contribute to the edge circadian score

In [10]:
# get all the paths that contribute to the final score (traverse a gene in CircaDB)
score['score_details'][score['score_details'].whether_in_circadb == True]

,source_node,target_node,metapath,node_ids,node_names,rel_ids,rel_names,gene_symbol,whether_in_circadb,circadian_tissue,non_circadian_tissue,importance_score
1,Flurbiprofen,rheumatoid arthritis,CbGbCpD,"16339,36580,37708,31624","Flurbiprofen,PTGS1,Salsalate,rheumatoid arthritis","901577,302279,326840","BINDS_CbG,BINDS_CbG,PALLIATES_CpD",PTGS1,True,Liver,"Aorta,Artery Coronary,Artery Tibial,Colon,Esop...",1.373082
4,Flurbiprofen,rheumatoid arthritis,CbGbCpD,"16339,36580,46014,31624","Flurbiprofen,PTGS1,Trisalicylate-choline,rheum...","901577,2134093,45886","BINDS_CbG,BINDS_CbG,PALLIATES_CpD",PTGS1,True,Liver,"Aorta,Artery Coronary,Artery Tibial,Colon,Esop...",0.970916
6,Flurbiprofen,rheumatoid arthritis,CbGbCpD,"16339,36580,46580,31624","Flurbiprofen,PTGS1,Oxaprozin,rheumatoid arthritis","901577,1879677,2240480","BINDS_CbG,BINDS_CbG,PALLIATES_CpD",PTGS1,True,Liver,"Aorta,Artery Coronary,Artery Tibial,Colon,Esop...",0.792749
10,Flurbiprofen,rheumatoid arthritis,CbGbCpD,"16339,36580,20448,31624","Flurbiprofen,PTGS1,Nabumetone,rheumatoid arthr...","901577,125767,2052632","BINDS_CbG,BINDS_CbG,PALLIATES_CpD",PTGS1,True,Liver,"Aorta,Artery Coronary,Artery Tibial,Colon,Esop...",0.686541
13,Flurbiprofen,rheumatoid arthritis,CbGbCpD,"16339,36580,45240,31624","Flurbiprofen,PTGS1,Diflunisal,rheumatoid arthr...","901577,1502545,1739798","BINDS_CbG,BINDS_CbG,PALLIATES_CpD",PTGS1,True,Liver,"Aorta,Artery Coronary,Artery Tibial,Colon,Esop...",0.560558
18,Flurbiprofen,rheumatoid arthritis,CbGbCpD,"16339,36580,34787,31624","Flurbiprofen,PTGS1,Tolmetin,rheumatoid arthritis","901577,2094235,960570","BINDS_CbG,BINDS_CbG,PALLIATES_CpD",PTGS1,True,Liver,"Aorta,Artery Coronary,Artery Tibial,Colon,Esop...",0.457694
23,Flurbiprofen,rheumatoid arthritis,CbGbCpD,"16339,36580,4689,31624","Flurbiprofen,PTGS1,Piroxicam,rheumatoid arthritis","901577,2001216,1266645","BINDS_CbG,BINDS_CbG,PALLIATES_CpD",PTGS1,True,Liver,"Aorta,Artery Coronary,Artery Tibial,Colon,Esop...",0.396375
30,Flurbiprofen,rheumatoid arthritis,CbGbCpD,"16339,36580,32527,31624","Flurbiprofen,PTGS1,Meloxicam,rheumatoid arthritis","901577,986154,1353379","BINDS_CbG,BINDS_CbG,PALLIATES_CpD",PTGS1,True,Liver,"Aorta,Artery Coronary,Artery Tibial,Colon,Esop...",0.328229
34,Flurbiprofen,rheumatoid arthritis,CbGbCpD,"16339,36580,31436,31624","Flurbiprofen,PTGS1,Naproxen,rheumatoid arthritis","901577,1005159,1184780","BINDS_CbG,BINDS_CbG,PALLIATES_CpD",PTGS1,True,Liver,"Aorta,Artery Coronary,Artery Tibial,Colon,Esop...",0.261836
37,Flurbiprofen,rheumatoid arthritis,CbGbCpD,"16339,36580,18254,31624","Flurbiprofen,PTGS1,Acetylsalicylic acid,rheuma...","901577,1377559,1888220","BINDS_CbG,BINDS_CbG,PALLIATES_CpD",PTGS1,True,Liver,"Aorta,Artery Coronary,Artery Tibial,Colon,Esop...",0.217103


All 12 paths traverse the same gene: PTGS1(5742)

## Example 2: theophylline (DB00277) and asthma (DOID:2841)

In [11]:
# Return score details of Flurbiprofen ~ rheumatoid arthritis
drug = 'DB00277'
disease = 'DOID:2841'
tissues = ['Aorta','Artery Coronary','Artery Tibial','Colon','Esophagus','Fat SQ','Fat Visceral','Heart Atrial','Liver',
           'Lung','Nerve Tibial','Pituitary','Thyroid']
score = cal.detail_edge_circa_score(drug, disease, tissues, circa_db, query_metapath = metapaths)

In [12]:
# edge circadian score of drug-disease pair in 13 tissues
score['edge_circa_score']

{'Aorta': 0.0,
 'Artery Coronary': 0.22099544141782215,
 'Artery Tibial': 0.0,
 'Colon': 0.0,
 'Esophagus': 0.28514488746640076,
 'Fat SQ': 0.0009090649629639946,
 'Fat Visceral': 0.0009090649629639946,
 'Heart Atrial': 0.043457446487494174,
 'Liver': 0.0,
 'Lung': 0.0,
 'Nerve Tibial': 0.0,
 'Pituitary': 0.0,
 'Thyroid': 0.2016689559253745}

In [13]:
# number of metapaths that contain gene
score['total_meta_count']

4

In [14]:
# show four metapaths
score['score_details'].metapath.value_counts()

CbGbCtD    70
CdGeAlD    17
CrCbGaD    11
CbGuDrD     3
Name: metapath, dtype: int64

In [15]:
# number of paths that traverse gene in CircaDB
score['total_path_count']

41

In [16]:
# number of paths that contribute to the final score, should equal the number above
sum(score['score_details'].whether_in_circadb == True)

41